In [ ]:
!pip install datasets

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import json
import re
import os
import shutil
import datasets

from sklearn.metrics import accuracy_score, f1_score
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset


# from transformers import BertTokenizer, TFBertForSequenceClassification
# from transformers import InputExample, InputFeatures
# # from deep_translator import GoogleTranslator
from datasets import Dataset

from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
# from sklearn.model_selection import train_test_split
# import torch

# from sklearn.metrics import accuracy_score
# os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'


In [ ]:
!wget "https://raw.githubusercontent.com/ancatache/LaRoSeDa/main/data_splitted/laroseda_train.json"
f = open('laroseda_train.json')
# returns JSON object as
# a dictionary
train_data = json.load(f)

# Closing file
f.close()

--2024-09-03 01:23:56--  https://raw.githubusercontent.com/ancatache/LaRoSeDa/main/data_splitted/laroseda_train.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4231664 (4.0M) [text/plain]
Saving to: ‘laroseda_train.json.2’

laroseda_train.json 100%[===================>]   4.04M  --.-KB/s    in 0.05s   

2024-09-03 01:23:56 (81.0 MB/s) - ‘laroseda_train.json.2’ saved [4231664/4231664]



In [ ]:
!wget "https://raw.githubusercontent.com/ancatache/LaRoSeDa/main/data_splitted/laroseda_test.json"
f = open('laroseda_test.json')
# returns JSON object as
# a dictionary
test_data = json.load(f)

# Closing file
f.close()


--2024-09-03 01:23:56--  https://raw.githubusercontent.com/ancatache/LaRoSeDa/main/data_splitted/laroseda_test.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1025519 (1001K) [text/plain]
Saving to: ‘laroseda_test.json.2’

laroseda_test.json. 100%[===================>]   1001K  --.-KB/s    in 0.04s   

2024-09-03 01:23:57 (24.5 MB/s) - ‘laroseda_test.json.2’ saved [1025519/1025519]



In [ ]:
#  # Add a new padding token if not already present
# if tokenizer.pad_token is None:
#         tokenizer.add_special_tokens({'pad_token': '[PAD]'})

# # Set the padding token ID in the model configuration
# model.config.pad_token_id = tokenizer.pad_token_id



In [ ]:
def prep_doc(doc):
    cleaned = list()
    for i in doc:
        rev = i['title'] + ' . ' + i['content']
        # rev=i['content']
        # rev = re.sub('(?<! )(?=[.,!?()])|(?<=[.,!?()])(?! )', r' ', rev)
        rev = rev.lower()
        rev = re.sub(r'\s+', ' ', rev)
        rev = re.sub(r'(?<! )(?=[.,!?()"\'/:;/\\\[^\]])|(?<=[.,!?()"\'/:;/\\\[^\]])(?! )', r' ', rev)
        cleaned.append(rev)
    return cleaned


def read_data(set):
    pos_data = list()
    neg_data = list()
    for i in set:
        # print(i)
        match i['starRating']:
            case '1':
                neg_data.append(i)
            case '2':
                neg_data.append(i)
            case '4':
                pos_data.append(i)
            case '5':
                pos_data.append(i)
    return neg_data, pos_data


def read_docs():
    f = open('laroseda_train.json')
    # returns JSON object as
    # a dictionary
    train_data = json.load(f)
    f.close()

    f = open('laroseda_test.json')
    # returns JSON object as
    # a dictionary
    test_data = json.load(f)
    # Closing file
    f.close()

    train_neg, train_pos = read_data(train_data['reviews'])
    test_neg, test_pos = read_data(test_data['reviews'])

    train_pos_clean = prep_doc(train_pos)
    train_neg_clean = prep_doc(train_neg)
    test_pos_clean = prep_doc(test_pos)
    test_neg_clean = prep_doc(test_neg)

    return train_pos_clean, train_neg_clean, test_pos_clean, test_neg_clean


# def compute_metrics(eval_pred):
#     logits, labels = eval_pred
#     predictions = np.argmax(logits, axis=-1)
#     accuracy = accuracy_score(labels, predictions)
#     f1 = f1_score(labels, predictions, average='weighted')

#     return {"accuracy": accuracy,"f1_score":f1}

#
# def preprocess_function(examples):
#     return tokenizer(examples["text"], truncation=True)




# def run_setup(model_identifier,train_texts,test_texts):

#     train_labels = [1] * len(train_pos_clean) + [0] * len(train_neg_clean)
#     test_labels = [1] * len(test_pos_clean) + [0] * len(test_neg_clean)

#     # Load pre-trained model and tokenizer
#     # model_identifier = "dumitrescustefan/bert-base-romanian-cased-v1"
#     # model_identifier = 'bert-base-multilingual-cased'
#     tokenizer = AutoTokenizer.from_pretrained(model_identifier)
#     model = AutoModelForSequenceClassification.from_pretrained(model_identifier, num_labels=2)  # Adjust num_labels

#     # Add a new padding token if not already present
#     if tokenizer.pad_token is None:
#         tokenizer.add_special_tokens({'pad_token': '[PAD]'})

#     # Set the padding token ID in the model configuration
#     model.config.pad_token_id = tokenizer.pad_token_id




#     train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=200)
#     test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=200)

#     # Create Hugging Face Dataset objects using tokenized data
#     train_dataset = Dataset.from_dict({
#         'input_ids': train_encodings['input_ids'],
#         'attention_mask': train_encodings['attention_mask'],
#         'label': train_labels
#     })

#     test_dataset = Dataset.from_dict({
#         'input_ids': test_encodings['input_ids'],
#         'attention_mask': test_encodings['attention_mask'],
#         'label': test_labels
#     })

#     # Set the format of the datasets for PyTorch
#     train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
#     test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

#     torch.save(model.state_dict(), 'model.pth')

#     training_args = TrainingArguments(
#         output_dir="./results",
#         save_strategy="no",  # Disable saving completely during training
#         num_train_epochs=3,
#         per_device_train_batch_size=1,
#         per_device_eval_batch_size=1,
#         eval_strategy="steps",  # Update eval_strategy
#         logging_dir="./logs",
#     )


#     trainer = Trainer(
#         model=model,
#         args=training_args,
#         train_dataset=train_dataset,
#         eval_dataset=test_dataset,
#         tokenizer=tokenizer,
#         compute_metrics=compute_metrics  # Add this line
#     )

#     return trainer, model, tokenizer





In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [ ]:
# def compute_metrics(eval_pred):
#     logits, labels = eval_pred
#     print("Logits shape:", logits.shape)
#     predictions = np.argmax(logits, axis=-1)
#     accuracy = accuracy_score(labels, predictions)
#     f1 = f1_score(labels, predictions, average='weighted')
#     return {"accuracy": accuracy, "f1_score": f1}

# def compute_metrics(p):
#     logits = p.predictions[0]  # Access the first element of the tuple, which contains the logits
#     preds = logits.argmax(-1)  # Apply argmax to get the predicted class
#     labels = p.label_ids

#     accuracy = accuracy_score(labels, preds)
#     f1 = f1_score(labels, preds, average="weighted")  # Use weighted F1-score for imbalanced datasets

#     return {"accuracy": accuracy, "f1": f1}

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    if isinstance(logits, tuple):
        logits = logits[0]  # Unpack the first element of the tuple
    print("Logits shape:", logits.shape)
    predictions = np.argmax(logits, axis=-1)
    accuracy = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average='weighted')
    return {"accuracy": accuracy, "f1_score": f1}

def run_test(model_identifier, train_texts, test_texts, train_labels, test_labels):
    # Load pre-trained model and tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_identifier)

    # Add a new padding token if not already present
    if tokenizer.pad_token is None:
        tokenizer.add_special_tokens({'pad_token': '[PAD]'})

    model = AutoModelForSequenceClassification.from_pretrained(model_identifier, num_labels=2)

    # Resize token embeddings after adding new special tokens
    model.resize_token_embeddings(len(tokenizer))

    # Set the padding token ID in the model configuration
    model.config.pad_token_id = tokenizer.pad_token_id

    # Tokenize the texts with padding and truncation
    train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=200)
    test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=200)

    train_dataset = Dataset.from_dict({
        'input_ids': train_encodings['input_ids'],
        'attention_mask': train_encodings['attention_mask'],
        'label': train_labels
    })

    test_dataset = Dataset.from_dict({
        'input_ids': test_encodings['input_ids'],
        'attention_mask': test_encodings['attention_mask'],
        'label': test_labels
    })

    # Set the format of the datasets for PyTorch
    train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
    test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

    training_args = TrainingArguments(
        output_dir="./results",
        save_strategy="no",
        num_train_epochs=3,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=32,
        evaluation_strategy="epoch",
        logging_dir="./logs",
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )

    trainer.train()

    # Evaluate the model
    eval_result = trainer.evaluate()
    print(f"Evaluation results: {eval_result}")

    return eval_result



In [ ]:
# print(train_pos_clean[6])

In [ ]:

# Load and preprocess data
train_pos_clean, train_neg_clean, test_pos_clean, test_neg_clean = read_docs()

train_texts = train_pos_clean + train_neg_clean
train_labels = [1] * len(train_pos_clean) + [0] * len(train_neg_clean)

test_texts = test_pos_clean + test_neg_clean
test_labels = [1] * len(test_pos_clean) + [0] * len(test_neg_clean)

names = [
        #  'dumitrescustefan/t5-v1_1-base-romanian'
        #  'racai/distilbert-base-romanian-uncased', #--
        #  'racai/distilbert-base-romanian-cased', #--
        #  'dumitrescustefan/bert-base-romanian-uncased-v1', #---
        #  'dumitrescustefan/bert-base-romanian-cased-v1', #---
        #  'xlm-roberta-base' #---
        #  'bert-base-multilingual-cased',  #---
        #  'bert-base-multilingual-uncased',  #---
        #  'gpt2',  # The standard GPT-2 model identifier #---
        #  'DGurgurov/xlm-r_romanian_sentiment' #---
         'readerbench/RoGPT2-base'

]

 #  'OpenLLM-Ro/RoLlama2-7b-Base',
 #  'OpenLLM-Ro/RoGemma-7b-Instruct',

# Run the test with GPT-2
eval_result = run_test(names[0], train_texts, test_texts, train_labels, test_labels)


tokenizer_config.json:   0%|          | 0.00/563 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/814 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/985k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/542k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/357 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/510M [00:00<?, ?B/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at readerbench/RoGPT2-base and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1 Score
1,0.087600,0.130889,0.970333,0.970315
2,0.038700,0.096660,0.979000,0.979000
3,0.017700,0.095963,0.982667,0.982666


Logits shape: (3000, 2)
Logits shape: (3000, 2)
Logits shape: (3000, 2)


Logits shape: (3000, 2)
Evaluation results: {'eval_loss': 0.09596286714076996, 'eval_accuracy': 0.9826666666666667, 'eval_f1_score': 0.9826658962620561, 'eval_runtime': 18.5171, 'eval_samples_per_second': 162.013, 'eval_steps_per_second': 5.076, 'epoch': 3.0}


In [ ]:
breq

In [ ]:
#baaaaaaaaaaaaaaazic

from sklearn.metrics import accuracy_score, f1_score
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset

# Define model and tokenizer
model_name = "racai/distilbert-base-romanian-uncased"
# model_name = "dumitrescustefan/t5-v1_1-base-romanian"
# model_name ='DGurgurov/xlm-r_romanian_sentiment'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)  # Binary classification

# Add a new padding token if not already present
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))
model.config.pad_token_id = tokenizer.pad_token_id

# Data loading (Replace these with your actual data loading code)
train_pos_clean, train_neg_clean, test_pos_clean, test_neg_clean = read_docs()

# Prepare datasets
train_texts = train_pos_clean + train_neg_clean
train_labels = [1] * len(train_pos_clean) + [0] * len(train_neg_clean)
test_texts = test_pos_clean + test_neg_clean
test_labels = [1] * len(test_pos_clean) + [0] * len(test_neg_clean)

# Tokenizing the data
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=256)  # Specify max_length

train_dataset = Dataset.from_dict({"text": train_texts, "label": train_labels})
test_dataset = Dataset.from_dict({"text": test_texts, "label": test_labels})

train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

def compute_metrics(p):
    logits = p.predictions[0]  # Access the first element of the tuple, which contains the logits
    preds = logits.argmax(-1)  # Apply argmax to get the predicted class
    labels = p.label_ids

    accuracy = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average="weighted")  # Use weighted F1-score for imbalanced datasets

    return {"accuracy": accuracy, "f1": f1}

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    save_steps=10_000,
    save_total_limit=2,
    remove_unused_columns=False,
    push_to_hub=False,
)

# Initialize Trainer with the compute_metrics function
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,  # Add this to calculate metrics
)

# Train the model
trainer.train()

# Evaluate the model
eval_results = trainer.evaluate()
print(f"Evaluation results: {eval_results}")

# Prediction function
def predict(texts):
    # Move the model to GPU if available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    # Tokenize inputs and move tensors to the same device as the model
    inputs = tokenizer(texts, padding=True, truncation=True, max_length=256, return_tensors="pt")
    inputs = {key: val.to(device) for key, val in inputs.items()}

    # Ensure the model is in evaluation mode
    model.eval()

    with torch.no_grad():  # Disable gradient calculation to save memory
        outputs = model(**inputs)

    predictions = torch.argmax(outputs.logits, dim=-1)
    return predictions

# Example usage
new_texts = ["Acest film este fantastic!", "Nu mi-a placut acest film."]
predictions = predict(new_texts)
print(predictions)  # Output will be the predicted labels


In [ ]:
breq



In [ ]:
# Install required libraries (uncomment if not already installed)
# !pip install torch transformers

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# Define model and tokenizer
model_name = "OpenLLM-Ro/RoLlama2-7b-Base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)  # Use causal language model for generation

# Function to generate text
def generate_text(prompt, max_length=50):
    inputs = tokenizer(prompt, return_tensors="pt")
    # Move tensors to GPU if available
    if torch.cuda.is_available():
        inputs = {key: val.to('cuda') for key, val in inputs.items()}
        model.to('cuda')

    outputs = model.generate(
        **inputs,
        max_length=max_length,
        num_return_sequences=1,  # Generate one sequence
        no_repeat_ngram_size=2,  # Prevent repeating n-grams
        early_stopping=True  # Stop early if the model generates an end token
    )

    # Decode the generated text
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_text

# Example usage
prompt = "Explain the significance of the theory of relativity in modern physics."
generated_response = generate_text(prompt)
print(generated_response)


In [ ]:
# Example usage
prompt = "tu ma intelegi?"
generated_response = generate_text(prompt)
print(generated_response)

In [ ]:
breq


In [ ]:
# Install required libraries (uncomment if not already installed)
# !pip install torch transformers datasets accelerate

import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Define model and tokenizer
model_name = "OpenLLM-Ro/RoLlama2-7b-Base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)  # Binary classification

# Add a new padding token if not already present
if tokenizer.pad_token is None:
  tokenizer.add_special_tokens({'pad_token': '[PAD]'})
model.config.pad_token_id = tokenizer.pad_token_id

# Data loading (Replace these with your actual data loading code)
train_pos_clean, train_neg_clean, test_pos_clean, test_neg_clean = read_docs()

# Prepare datasets
train_texts = train_pos_clean + train_neg_clean
train_labels = [1] * len(train_pos_clean) + [0] * len(train_neg_clean)
test_texts = test_pos_clean + test_neg_clean
test_labels = [1] * len(test_pos_clean) + [0] * len(test_neg_clean)

# Tokenizing the data
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)  # Adjust max_length

# Prepare dataset in PyTorch format
from torch.utils.data import Dataset

class TextDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = 128

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(text, truncation=True, padding='max_length', max_length=self.max_length, return_tensors='pt')
        return {**{key: torch.squeeze(val) for key, val in encoding.items()}, 'label': torch.tensor(label)}

train_dataset = TextDataset(train_texts, train_labels)
test_dataset = TextDataset(test_texts, test_labels)

# Prediction function
def predict(texts):
    inputs = tokenizer(texts, padding=True, truncation=True, max_length=128, return_tensors="pt")  # Adjust max_length
    outputs = model(**inputs)
    predictions = torch.argmax(outputs.logits, dim=-1)
    return predictions

# Example usage
new_texts = ["This movie was fantastic!", "I did not like the film."]
predictions = predict(new_texts)
print(predictions)  # Output will be the predicted labels


In [ ]:
def predict(texts):
    # Move the model to GPU if available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    # Tokenize inputs and move tensors to the same device as the model
    inputs = tokenizer(texts, padding=True, truncation=True, max_length=256, return_tensors="pt")
    inputs = {key: val.to(device) for key, val in inputs.items()}

    # Ensure the model is in evaluation mode
    model.eval()

    with torch.no_grad():  # Disable gradient calculation to save memory
        outputs = model(**inputs)

    predictions = torch.argmax(outputs.logits, dim=-1)
    return predictions

# Example usage
new_texts = ["This movie was fantastic!", "I did not like the film."]
predictions = predict(new_texts)
print(predictions)  # Output will be the predicted labels



In [ ]:

# Example usage
new_texts = test_dataset.to_list()
predictions = predict(new_texts)
print(predictions)  # Output will be the predicted labels


In [ ]:
# Install required libraries (uncomment if not already installed)
# !pip install torch transformers datasets accelerate

import os
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset

# Set environment variable for CUDA memory management
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

# Define model and tokenizer
model_name = "OpenLLM-Ro/RoLlama2-7b-Base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)  # Binary classification

# Data loading (Replace these with your actual data loading code)
train_pos_clean, train_neg_clean, test_pos_clean, test_neg_clean = read_docs()

# Prepare datasets
train_texts = train_pos_clean + train_neg_clean
train_labels = [1] * len(train_pos_clean) + [0] * len(train_neg_clean)
test_texts = test_pos_clean + test_neg_clean
test_labels = [1] * len(test_pos_clean) + [0] * len(test_neg_clean)

# Tokenizing the data
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)  # Adjust max_length

train_dataset = Dataset.from_dict({"text": train_texts, "label": train_labels})
test_dataset = Dataset.from_dict({"text": test_texts, "label": test_labels})

train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

# Training arguments with mixed precision and gradient accumulation
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    per_device_train_batch_size=1,  # Batch size
    per_device_eval_batch_size=1,  # Batch size
    num_train_epochs=3,
    save_steps=10_000,
    save_total_limit=2,
    fp16=True,  # Mixed precision training
    gradient_accumulation_steps=4,  # Accumulate gradients over 4 steps
    remove_unused_columns=False,
    push_to_hub=False,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

# Clear CUDA cache
torch.cuda.empty_cache()

# Train the model
trainer.train()

# Evaluate the model
eval_results = trainer.evaluate()
print(f"Evaluation results: {eval_results}")

# Prediction function
def predict(texts):
    inputs = tokenizer(texts, padding=True, truncation=True, max_length=128, return_tensors="pt")  # Adjust max_length
    outputs = model(**inputs)
    predictions = torch.argmax(outputs.logits, dim=-1)
    return predictions

# Example usage
new_texts = ["This movie was fantastic!", "I did not like the film."]
predictions = predict(new_texts)
print(predictions)  # Output will be the predicted labels


In [ ]:

train_pos_clean, train_neg_clean, test_pos_clean, test_neg_clean = read_docs()

train_texts = train_pos_clean + train_neg_clean
train_labels = [1] * len(train_pos_clean) + [0] * len(train_neg_clean)

test_texts = test_pos_clean + test_neg_clean
test_labels = [1] * len(test_pos_clean) + [0] * len(test_neg_clean)


In [ ]:

names=[
      #  'racai/distilbert-base-romanian-uncased',
      #  'dumitrescustefan/bert-base-romanian-cased-v1',
      #  'bert-base-multilingual-cased',
      #  'OpenLLM-Ro/RoLlama2-7b-Base'
       'OpenLLM-Ro/RoMistral-7b-Instruct'
       ]

trainer, model, tokenizer = run_setup(names[0],train_texts, test_texts)

#
# model = AutoModelForSequenceClassification.from_pretrained('nico-dv/sentiment-analysis-model', num_labels=2)  # Adjust num_labels
#
# # nico-dv/sentiment-analysis-model
#
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_dataset,
#     eval_dataset=test_dataset,
#     tokenizer=tokenizer,
#     compute_metrics=compute_metrics  # Add this line
# )
#
# eval_result = trainer.evaluate()
# print(f"Evaluation results: {eval_result}")

In [ ]:
PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True

In [ ]:

eval_result = run_train(trainer)


In [ ]:

# Prediction function
def predict(texts):
    inputs = tokenizer(texts, padding=True, truncation=True, max_length=256, return_tensors="pt")  # Specify max_length
    outputs = model(**inputs)
    predictions = torch.argmax(outputs.logits, dim=-1)
    return predictions
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=256)  # Specify max_length

In [ ]:
trainer.train()

# Evaluate the model
eval_results = trainer.evaluate()
print(f"Evaluation results: {eval_results}")

# Prediction function
def predict(texts):
    inputs = tokenizer(texts, padding=True, truncation=True, max_length=256, return_tensors="pt")  # Specify max_length
    outputs = model(**inputs)
    predictions = torch.argmax(outputs.logits, dim=-1)
    return predictions

# Example usage
new_texts = ["This movie was fantastic!", "I did not like the film."]
predictions = predict(new_texts)
print(predictions)  # Output will be the predicted labels

In [ ]:
torch.cuda.empty_cache()
